In [1]:
import re
import os
import pickle
import numpy as np
import logging
import torch
import math
import time
from torch.utils.data import Dataset
from utils.utils import newsample, getId2idx, tokenize, getVocab, my_collate, Partition_Sampler
from data.configs.demo import config
from torch.utils.data import DataLoader
from collections import defaultdict

from transformers import BertTokenizer,BertModel
from utils.MIND import MIND

logger = logging.getLogger(__name__)

In [2]:
# config.reducer = 'bm25'
# config.reducer = 'bow'
# config.reducer = 'matching'

# config.scale = 'large'
path = config.path + 'MIND/MINDdemo_train/'
a = MIND(config, path + 'news.tsv', path + 'behaviors.tsv')

[2021-08-28 17:20:49,949] INFO (utils.MIND) loading cached user behavior from data/cache/bert/MINDdemo_train/10/behaviors..pkl
[2021-08-28 17:20:49,968] INFO (utils.MIND) loading cached news tokenization from data/cache/bert/MINDdemo_train/news.pkl
[2021-08-28 17:20:51,288] INFO (utils.MIND) reducing news of ../../../Data/MIND/MINDdemo_train/news.tsv...
[2021-08-28 17:20:51,292] INFO (utils.utils) unmasking at least k...
[2021-08-28 17:20:51,963] INFO (utils.utils) deduplicating...


In [ ]:
t1 = time.time()
a[10]
t2 = time.time()
t2-t1

In [ ]:
dic = pickle.load(open('data/cache/bert/MINDdemo_dev/news.pkl', 'rb'))

news = dic['encoded_news']
subwords = dic['subwords']

In [3]:
loader1 = DataLoader(a, batch_size=1, pin_memory=False, num_workers=0, drop_last=False, shuffle=False)
records1 = list(loader1)

In [6]:
records1[0]['cdd_subword_prefix']

tensor([[[[1., 0., 0.,  ..., 0., 0., 0.],
          [0., 1., 0.,  ..., 0., 0., 0.],
          [0., 0., 1.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]],

         [[1., 0., 0.,  ..., 0., 0., 0.],
          [0., 1., 0.,  ..., 0., 0., 0.],
          [0., 0., 1.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]],

         [[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]],

         [[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          